Route changed e nossa target

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [ ]:
df_train=pd.read_csv('./data/clean_train.csv', index_col=None)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1996516 entries, 1912282 to 14413237
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unnamed: 0              int64  
 1   tr_id                   float64
 2   tr_dst                  float64
 3   total_probes_sent       float64
 4   total_replies_last_hop  float64
 5   route_changed           float64
 6   seconds_since_start     float64
 7   mean_rtt                float64
dtypes: float64(7), int64(1)
memory usage: 137.1 MB


In [24]:
df_test=pd.read_csv('./data/clean_test.csv', index_col=None)
df_test=df_test.sample(frac=0.1,random_state=12345)

In [25]:
X=df_train.drop('route_changed',axis=1)
y=df_train['route_changed']

seed_list=np.random.choice(range(1, 1000000), size=100, replace=False)

In [26]:
print(X.shape)
print(y.shape)

(1996516, 7)
(1996516,)


In [27]:
def K_Fold_XGBoost(X, y,n_splits=10,random_state=12345,n_estimators=50, n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= xgb.XGBClassifier(n_estimators=n_estimators, max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

#def K_Fold_Forest(X, y,n_splits=10,random_state=12345,n_estimators=20,n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= RandomForestClassifier(n_estimators=n_estimators,max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Tree(X, y,n_splits=10,random_state=12345,n_depth=10):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= DecisionTreeClassifier(max_depth=n_depth, random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

def K_Fold_Logistic(X, y,n_splits=10,random_state=12345):
    cv = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    model= LogisticRegression(random_state=random_state)
    
    acc_list=[]
    precision_list=[]
    recall_list=[]
    f1_list=[]
    
    for fold, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X.iloc[train],y.iloc[train])
        prediction=model.predict(X.iloc[test])
        acc_list.append(accuracy_score(y.iloc[test],prediction))
        precision_list.append(precision_score(y.iloc[test],prediction,zero_division=0))
        recall_list.append(recall_score(y.iloc[test],prediction))
        f1_list.append(f1_score(y.iloc[test],prediction))

    return [np.mean(acc_list),np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)]

In [31]:
K_Fold_XGBoost(X,y,10,12345,21,None)

[np.float64(0.9814221373616698),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [32]:
K_Fold_Logistic(X,y)

[np.float64(0.9814231391097202),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [33]:
results=K_Fold_Tree(X,y)

In [19]:
print(results)

[np.float64(0.981333482932666), np.float64(0.031760179391758334), np.float64(0.00016177594195640037), np.float64(0.0003218313281671173)]


In [ ]:
def data_collection_tree(seed_list,X,y,n=30):
    df=pd.DataFrame()
    for i in range(1,n+1):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_Tree(X,y,10,seed,i)
            result_list.append(result)
        df[i]=result_list
        print(i)
    print('Done')
    return df

def data_collection_xgboost(seed_list,X,y,n=51):
    df=pd.DataFrame()
    for i in range(1,n+1,5):
        result_list=[]
        for seed in seed_list:
            result=K_Fold_XGBoost(X,y,10,seed,i,None)
            result_list.append(result)
        df[i]=result_list
        print(i)
    print('Done')
    return df

def data_collection_logistic(seed_list, X, y):
    result_list=[]
    for seed in seed_list:
        result=K_Fold_Logistic(X,y,10,seed)
        result_list.append(result)
    print('Done')
    return result_list

In [21]:
results_tree=data_collection_tree(seed_list,X,y)
results_tree_df=pd.DataFrame(results_tree, columns=['acc','precision','recall','F1'])

/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

1


/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/matheus-henrique/miniforge3/envs/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

KeyboardInterrupt: 

In [ ]:
results_tree_df.to_csv('./data/result_tree.csv',index=False)

In [ ]:
results_xgboost=data_collection_xgboost(seed_list,X,y)
results_xgboost_df=pd.DataFrame(results_xgboost, columns=['acc','precision','recall','F1'])

In [ ]:
results_xgboost_df.to_csv('./data/result_xgboost.csv',index=False)

In [30]:
results_logistic=data_collection_logistic(seed_list,X,y)
results_logistic_df=pd.DataFrame(results_logistic, columns=['acc','precision','recall','F1'])

KeyboardInterrupt: 

In [ ]:
results_logistic_df.to_csv('./data/result_logistic.csv',index=False)